In [59]:
%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell

from data_loader import get_pdf_filepaths, load_page_and_line_indexes, load_split_data, load_raw_indexes_list
from pdf_reader import parse_pdf
from data_cleaner import clean_initial_indexes, add_split_data
from data_saver import save_page_and_line_indexes, save_split_data, save_raw_indexes_list
from data_transformer import (
  get_candidates_and_frequencies, 
  add_frequencies_column, 
  add_is_in_toc, 
  add_importance,
  add_position_in_context,
  add_is_named_entity,
  add_length_of_word,
  add_is_named_author,
  add_tfidf,
  get_raw_indexes_list,
  add_is_in_index,
  aggregate_by_candidate
)


#InteractiveShell.ast_node_interactivity = "all"

# Configuration vars

In [2]:
# Configuration vars
DATA_DIR_PATH = "../data/"
PDF_SOURCE_DIR_PATH = DATA_DIR_PATH + "pdf/"
PROCESSED_DATA_DIR_PATH = DATA_DIR_PATH + "processed/"

# Load data

In [3]:
file_paths = get_pdf_filepaths(PDF_SOURCE_DIR_PATH)
file_paths

['/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2017_Book_TheDataScienceDesignManual.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/morris_07.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/lyons_96.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/2015_Book_LinearAlgebra.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/christensen_04.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/keefe_00.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/linnebo_18.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/oliver_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/jenkins_08.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/zimmermann_13.pdf',
 '/Users/bertagrim/dev_projects/final_project/index_generator/data/pdf/shapiro_9

# Raw line and page data


In [26]:

# TODO: Does not work with file_paths[0] and file_paths[2]
for file_path in file_paths[3:4]:
    raw_line_and_page_indexes = parse_pdf(file_path)

    line_and_page_indexes = clean_initial_indexes(raw_line_and_page_indexes)

  # line_and_page_indexes['file_name']
  # line_and_page_indexes['by_line'].head()
  # line_and_page_indexes['by_page'].head()

    save_page_and_line_indexes(
     processed_data_dir_path=PROCESSED_DATA_DIR_PATH, 
     line_and_page_indexes=line_and_page_indexes
    )


# Data set split


In [61]:
for file_path in file_paths[3:4]:
  line_and_page_indexes = load_page_and_line_indexes(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
  with_split_data = add_split_data(
    file_path=file_path,
    line_and_page_indexes=line_and_page_indexes
  )

  # with_split_data['by_page_body'].head()
  
  save_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    split_data=with_split_data  
  )
 


# Prepare input data frames

In [60]:
file_path = file_paths[3]

split_data = load_split_data(
    processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
    pdf_filepath=file_path
  )
# split_data['by_line_toc'].head()

(candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

# for file_path in file_paths[3:4]:
#   split_data = load_split_data(
#     processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
#     pdf_filepath=file_path
#   )
#   split_data['by_line_toc'].head()

#   (candidates_df, freq_ngrams) = get_candidates_and_frequencies(split_data)

#   candidates_df.head()

#   with_frequencies = add_frequencies_column(
#     by_pages_body_df=split_data['by_page_body'],
#     candidates_df=candidates_df,
#     freq_ngrams=freq_ngrams
#   )

#   with_frequencies.head()

candidates_df.head(30)

,candidate_keyword,clean_context,raw_context,POS
0,chapter,[chapter],chapter,NOUN
1,linear,"[linear, algebra, every, day, life]",linear algebra in every day life,PROPN
2,algebra,"[linear, algebra, every, day, life]",linear algebra in every day life,PROPN
3,every,"[linear, algebra, every, day, life]",linear algebra in every day life,DET
4,day,"[linear, algebra, every, day, life]",linear algebra in every day life,NOUN
5,life,"[linear, algebra, every, day, life]",linear algebra in every day life,NOUN
6,linear algebra,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK
7,algebra every,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK
8,every day,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK
9,day life,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK


In [7]:
# candidates_df[candidates_df['candidate_keyword'] == 'socio-cultural']
#candidates_df[candidates_df['candidate_keyword'] == 'socio']

,candidate_keyword,clean_context,raw_context,POS


In [62]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [63]:
# with_frequencies[500:510]
# len(freq_ngrams)

In [64]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

with_is_in_toc.head(30)


,candidate_keyword,clean_context,raw_context,POS,freq,is_in_toc
0,chapter,[chapter],chapter,NOUN,0.000518,0
1,linear,"[linear, algebra, every, day, life]",linear algebra in every day life,PROPN,0.002786,1
2,algebra,"[linear, algebra, every, day, life]",linear algebra in every day life,PROPN,0.000648,1
3,every,"[linear, algebra, every, day, life]",linear algebra in every day life,DET,0.001888,1
4,day,"[linear, algebra, every, day, life]",linear algebra in every day life,NOUN,0.000046,1
5,life,"[linear, algebra, every, day, life]",linear algebra in every day life,NOUN,0.000046,1
6,linear algebra,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK,0.000435,0
7,algebra every,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK,0.000037,0
8,every day,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK,0.000046,0
9,day life,"[linear, algebra, every, day, life]",linear algebra in every day life,CHUNK,0.000046,0


In [65]:
with_importance = add_importance(with_is_in_toc)



In [66]:
with_importance.head(30)

,candidate_keyword,clean_context,raw_context,POS,freq,is_in_toc,importance
0,chapter,chapter,chapter,NOUN,0.000518,0,-0.162285
1,linear,linear algebra every day life,linear algebra in every day life,PROPN,0.002786,1,0.541475
2,algebra,linear algebra every day life,linear algebra in every day life,PROPN,0.000648,1,0.541475
3,every,linear algebra every day life,linear algebra in every day life,DET,0.001888,1,0.541475
4,day,linear algebra every day life,linear algebra in every day life,NOUN,0.000046,1,0.541475
5,life,linear algebra every day life,linear algebra in every day life,NOUN,0.000046,1,0.541475
6,linear algebra,linear algebra every day life,linear algebra in every day life,CHUNK,0.000435,0,0.541475
7,algebra every,linear algebra every day life,linear algebra in every day life,CHUNK,0.000037,0,0.541475
8,every day,linear algebra every day life,linear algebra in every day life,CHUNK,0.000046,0,0.541475
9,day life,linear algebra every day life,linear algebra in every day life,CHUNK,0.000046,0,0.541475


In [67]:
with_position_in_context = add_position_in_context(with_importance)

In [68]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [69]:
with_is_named_entity.head(30)

,candidate_keyword,clean_context,raw_context,POS,freq,is_in_toc,importance,position_in_context,is_named_entity
0,chapter,chapter,chapter,NOUN,0.000518,0,-0.162285,0.000000,0
1,linear,linear algebra every day life,linear algebra in every day life,PROPN,0.002786,1,0.541475,0.000000,1
2,algebra,linear algebra every day life,linear algebra in every day life,PROPN,0.000648,1,0.541475,0.200000,1
3,every,linear algebra every day life,linear algebra in every day life,DET,0.001888,1,0.541475,0.600000,1
4,day,linear algebra every day life,linear algebra in every day life,NOUN,0.000046,1,0.541475,0.800000,1
5,life,linear algebra every day life,linear algebra in every day life,NOUN,0.000046,1,0.541475,1.000000,1
6,linear algebra,linear algebra every day life,linear algebra in every day life,CHUNK,0.000435,0,0.541475,0.000000,0
7,algebra every,linear algebra every day life,linear algebra in every day life,CHUNK,0.000037,0,0.541475,0.200000,0
8,every day,linear algebra every day life,linear algebra in every day life,CHUNK,0.000046,0,0.541475,0.600000,0
9,day life,linear algebra every day life,linear algebra in every day life,CHUNK,0.000046,0,0.541475,0.800000,0


In [70]:
with_length_of_word = add_length_of_word(candidates_df)

In [71]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_of_word,
  df_pages_biblio=split_data['by_page_biblio']
)

In [73]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_pages_body=split_data['by_page_body']
)

In [75]:
with_tfidf.describe().T

,count,mean,std,min,25%,50%,75%,max
freq,83537.0,0.002614,0.005065,0.000000,0.000037,0.000343,0.002175,0.020430
is_in_toc,83537.0,0.122425,0.327778,0.000000,0.000000,0.000000,0.000000,1.000000
importance,83537.0,0.266748,0.171825,-0.277741,0.152840,0.278597,0.390791,0.732903
position_in_context,83537.0,0.390919,0.304635,0.000000,0.125000,0.333333,0.631579,1.000000
is_named_entity,83537.0,0.086477,0.281068,0.000000,0.000000,0.000000,0.000000,1.000000
length,83537.0,6.868406,5.357519,1.000000,3.000000,5.000000,10.000000,35.000000
is_named_author,83537.0,0.002107,0.045852,0.000000,0.000000,0.000000,0.000000,1.000000
tfidf,83537.0,0.069366,0.145401,0.000000,0.000000,0.000000,0.066423,0.850750


In [52]:
split_data['by_page_body'].isnull().sum()

content              0
page_number          0
real_page_num        0
section_level_1      0
section_level_2     26
section_level_3    300
clean_content        0
dtype: int64

In [54]:
split_data['by_page_body'].fillna('',inplace=True)

In [55]:
split_data['by_page_body'].isnull().sum()

content            0
page_number        0
real_page_num      0
section_level_1    0
section_level_2    0
section_level_3    0
clean_content      0
dtype: int64

In [58]:
split_data['by_page_body'].groupby(
        ['section_level_1', 'section_level_2', 'section_level_3']
    )['clean_content'].apply(' '.join).reset_index()

section_level_1  \
0                             1 Linear Algebra in Every Day Life   
1                             1 Linear Algebra in Every Day Life   
2                             1 Linear Algebra in Every Day Life   
3                             1 Linear Algebra in Every Day Life   
4                             1 Linear Algebra in Every Day Life   
5                                                 10 Linear Maps   
6                                                 10 Linear Maps   
7                             11 Linear Forms and Bilinear Forms   
8                             11 Linear Forms and Bilinear Forms   
9                             11 Linear Forms and Bilinear Forms   
10                        12 Euclidean and Unitary Vector Spaces   
11                        12 Euclidean and Unitary Vector Spaces   
12                        12 Euclidean and Unitary Vector Spaces   
13                                    13 Adjoints of Linear Maps   
14                                    13 Adjoints of Linear Maps   
15                               14 Eigenvalues of Endomorphisms   
16                               14 Eigenvalues of Endomorphisms   
17                               14 Eigenvalues of Endomorphisms   
18         15 Polynomials and the Fundamental Theorem of Algebra   
19         15 Polynomials and the Fundamental Theorem of Algebra   
20    16 Cyclic Subspaces, Duality and the Jordan Canonical Form   
21    16 Cyclic Subspaces, Duality and the Jordan Canonical Form   
22    16 Cyclic Subspaces, Duality and the Jordan Canonical Form   
23    17 Matrix Functions and Systems  of Differential Equations   
24    17 Matrix Functions and Systems  of Differential Equations   
25                           18 Special Classes of Endomorphisms   
26                           18 Special Classes of Endomorphisms   
27                           18 Special Classes of Endomorphisms   
28                           19 The Singular Value Decomposition   
29                                 2 Basic Mathematical Concepts   
30                                 2 Basic Mathematical Concepts   
31                                 2 Basic Mathematical Concepts   
32          20 The Kronecker Product and Linear Matrix Equations   
33                                        3 Algebraic Structures   
34                                        3 Algebraic Structures   
35                                                    4 Matrices   
36                                                    4 Matrices   
37                   5 The Echelon Form and the Rank of Matrices   
38                   5 The Echelon Form and the Rank of Matrices   
39                   5 The Echelon Form and the Rank of Matrices   
40                                 6 Linear Systems of Equations   
41                                    7 Determinants of Matrices   
42                                    7 Determinants of Matrices   
43                                    7 Determinants of Matrices   
44  8 The Characteristic Polynomial  and Eigenvalues of Matrices   
45  8 The Characteristic Polynomial  and Eigenvalues of Matrices   
46  8 The Characteristic Polynomial  and Eigenvalues of Matrices   
47                                               9 Vector Spaces   
48                                               9 Vector Spaces   
49                                               9 Vector Spaces   
50                                               9 Vector Spaces   
51                                                    Appendix A   

                                                       section_level_2  \
0                                           1.1 The PageRank Algorithm   
1                           1.2 No Claim Discounting in Car Insurances   
2                                   1.3 Production Planning in a Plant   
3                                        1.4 Predicting Future Profits   
4                                               1.5 Circuit Simulation   


In [45]:
split_data['by_page_body']

,content,page_number,real_page_num,section_level_1,section_level_2,section_level_3,clean_content
0,"Chapter 1\n\nLinear Algebra in Every Day Life\n\nOne has to familiarize the student with actual questions from applications, so that he learns to deal with real world problems.1\nLothar Collatz (1910–1990)\n\n1.1 The PageRank Algorithm\nThe PageRank algorithm is a method to assess the “importance” of documents with mutual links, such as web pages, on the basis of the link structure.\nIt was developed by Sergei Brin and Larry Page, the founders of Google Inc., at Stanford University in the late 1990s.\nThe basic idea of the algorithm is the following:\nInstead of counting links, PageRank essentially interprets a link of page A to page\nB as a vote of page A for page B. PageRank then assesses the importance of a page by the number of received votes.\nPageRank also considers the importance of the page that casts the vote, since votes of some pages have a higher value, and thus also assign a higher value to the page they point to.\nImportant pages will be rated higher and thus lead to a higher position in the search results.2\nLet us describe (model) this idea mathematically.\nOur presentation uses ideas from the article [BryL06].\nFor a given set of web pages, every page k will be assigned an importance value xk ≥ 0.\nA page k is more important than a page j if xk > x j .\nIf a page k has a link to a page j, we say that page j has a backlink from page k.\nIn the above description these backlinks are the votes.\nAs an example, consider the following link structure:\n\n1 “Man muss den Lernenden mit konkreten Fragestellungen aus den Anwendungen vertraut machen, dass er lernt, konkrete Fragen zu behandeln.” of a text found in 2010 on http://www.google.de/corporate/tech.html.\n\n2 Translation\n\n© Springer International Publishing Switzerland 2015\nJ. Liesen and V. Mehrmann, Linear Algebra, Springer Undergraduate\nMathematics Series, DOI 10.1007/978-3-319-24346-7_1\n\n1",12.0,<FEFF>,1 Linear Algebra in Every Day Life,1.1 The PageRank Algorithm,NaN,chapter linear algebra every day life one familiarize student actual question application learns deal real world lothar collatz pagerank algorithm pagerank algorithm method ass importance document mutual link web page basis link structure developed sergei brin larry page founder google stanford university late basic idea algorithm following instead counting link pagerank essentially interprets link page page b vote page page pagerank ass importance page number received vote pagerank also considers importance page cast vote since vote page higher value thus also assign higher value page point important page rated higher thus lead higher position search let u describe model idea mathematically presentation us idea article given set web page every page k assigned importance value xk page k important page j xk x j page k link page j say page j backlink page description backlinks vote example consider following link structure man mus den lernenden mit konkreten fragestellungen au den anwendungen vertraut machen das er lernt konkrete fragen zu text found http translation springer international publishing switzerland liesen mehrmann linear algebra springer undergraduate mathematics series doi
1,"2\n\n1 Linear Algebra in Every Day Life\n\nHere the page 1 has links to the pages 2, 3 and 4, and a backlink from page 3.\nThe easiest approach to define importance of web pages is to count its backlinks; the more votes are cast for a page, the more important the page is.\nIn our example this gives the importance values x1 = 1, x2 = 3, x3 = 2, x4 = 3.\nThe pages 2 and 4 are thus the most important pages, and they are equally important.\nHowever, the intuition and also the above description from Google suggests that backlinks from important pages are more important for the value of a page than those from less important pages.\nThis idea can be modeled by defining xk as the sum of all importance values of the b

In [ ]:
with_tfidf.head()

In [ ]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [ ]:
raw_indexes_list

In [ ]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [ ]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [ ]:
# clean_indexes

In [ ]:
with_is_in_index = add_is_in_index(
  candidates_df=candidates_df,
  indexes_list=clean_indexes
)

In [ ]:
with_is_in_index.head()

In [ ]:
aggregate_by_candidate(
  candidates_df=with_is_in_index  
)

In [ ]:
# candidates_df[candidates_df['candidate_keyword'] == 'socio-cultural']
candidates_df[candidates_df['candidate_keyword'] == 'socio']

In [ ]:
with_frequencies = add_frequencies_column(
  by_pages_body_df=split_data['by_page_body'],
  candidates_df=candidates_df,
  freq_ngrams=freq_ngrams
)

In [ ]:
# with_frequencies[500:510]
# len(freq_ngrams)

In [ ]:
with_is_in_toc = add_is_in_toc(
  candidates_df=with_frequencies, 
  by_line_toc=split_data['by_line_toc']
)

with_is_in_toc.head()


In [ ]:
with_importance = add_importance(candidates_df=with_is_in_toc)

with_importance.head()

In [ ]:
with_position_in_context = add_position_in_context(candidates_df=with_importance)



In [ ]:
with_position_in_context[500:505]

In [ ]:
with_is_named_entity = add_is_named_entity(
  candidates_df=candidates_df,
  df_pages_body=split_data['by_page_body']
)

In [ ]:
with_is_named_entity.head()

In [ ]:
with_length_or_word = add_length_of_word(candidates_df)

In [ ]:
# with_length_or_word[500:503]

In [ ]:
with_is_named_author = add_is_named_author(
  candidates_df=with_length_or_word,
  df_cann_pages_biblio=split_data['by_page_biblio']
)

In [ ]:
with_is_named_author.head()

In [ ]:
with_tfidf = add_tfidf(
  candidates_df=with_is_named_author,
  df_cann_pages_body=split_data['by_page_body']
)

In [ ]:
with_tfidf.head()

In [ ]:
(raw_indexes_list, updated_by_line_index) = get_raw_indexes_list(
  df_cann_lines_index=split_data['by_line_index']
)

In [ ]:
raw_indexes_list

In [ ]:
save_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path,
  raw_indexes_list=raw_indexes_list
)

In [ ]:
clean_indexes = load_raw_indexes_list(
  processed_data_dir_path=PROCESSED_DATA_DIR_PATH,
  pdf_filepath=file_path
)

In [ ]:
# clean_indexes

In [ ]:
with_is_in_index = add_is_in_index(
  candidates_df=candidates_df,
  indexes_list=clean_indexes
)

In [ ]:
with_is_in_index.head()

In [ ]:
aggregate_by_candidate(
  candidates_df=with_is_in_index  
)